<a href="https://www.kaggle.com/code/chhelp/unit-6-advantage-actor-critic-a2c?scriptVersionId=143840295" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 创建虚拟屏幕

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

# 安装依赖

* panda-gym 包含机器人手臂环境
* stable-baselines3 SB3 深度学习环境

In [ ]:
!pip install stable-baselines3[extra]
!pip install gymnasium

In [ ]:
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

# 导包

In [ ]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

# PandaReachDense-v3 🦾

In [ ]:
# 创建环境
env_id = "PandaReachDense-v3"

env = gym.make(env_id)

# 状态空间和动作空间
s_size = env.observation_space.shape
a_size = env.action_space

The observation space **is a dictionary with 3 different elements**:
- `achieved_goal`: (x,y,z) position of the goal.
- `desired_goal`: (x,y,z) distance between the goal position and the current object position.
- `observation`: position (x,y,z) and velocity of the end-effector (vx, vy, vz).

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

The action space is a vector with 3 values:
- Control x, y, z movement

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action

## 归一化 observatin 和 rewards

In [ ]:
# 创建多个相同的环境实例 组成一个环境集合 这个环境集合可以同时运行多个环境实例
env = make_vec_env(env_id, n_envs=4)
# 对环境集合中的 observation 和 rewards 进行归一化处理
# env 是一个环境集合实例
# norm_obs=True 表示对观测值进行归一化处理
# norm_reward=True 表示对奖励值进行归一化处理
# clip_obs=10 表示对观测值进行截断处理 以避免出现过大或过小的观测值 (clip_obs=10 限制在[-10, 10] 的范围内)
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10)

## 创建 A2C(Advantage Actor-Critic) 模型

In [ ]:
# verbose 参数用于控制训练过程中的输出信息 当 verbose=1 时 模型会输出训练过程中的一些信息
model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)

# 训练 A2C Agent

In [ ]:
# model.learn(1_000_000)
# 这里我只是测试 所以 episodes 比较小
model.learn(1_000)

In [ ]:
# Save the model and  VecNormalize statistics when saving the agent
model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

# Evaluate the agent 📈
- Now that's our  agent is trained, we need to **check its performance**.
- Stable-Baselines3 provides a method to do that: `evaluate_policy`

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

# We need to override the render_mode
eval_env.render_mode = "rgb_array"

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load("a2c-PandaReachDense-v3")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")